In [50]:
import pandas as pd
import os
import glob

metric_files = glob.glob(f'{os.path.expanduser("~")}/experiments/*/logfile')

all_metrics = []

for i in range(len(metric_files)):
    file = metric_files[i]
    opened_file = open(file, 'r')

    accum_info = {}

    for line in opened_file.readlines():
        if 'exp_name' in line:
            exp_name = line.split('=')[-1].strip()

        if 'EPE3D' in line:
            epe3d = float(line.split(' ')[-1])

        if 'acc3d_strict' in line:
            acc3d_strict = float(line.split(' ')[-1])

        if 'acc3d_relax' in line:
            acc3d_relax = float(line.split(' ')[-1])

        if 'angle_error' in line:
            angle_error = float(line.split(' ')[-1])

        if 'outlier' in line:
            outlier = float(line.split(' ')[-1])

        if 'solver_time' in line:
            solver_time = float(line.split(' ')[-1])

    accum_info = {"exp_name" : exp_name,
                  "epe3d" : epe3d,
                  "acc3d_strict" : acc3d_strict,
                  "acc3d_relax" : acc3d_relax,
                  "angle_error" : angle_error,
                  "outlier" : outlier,
                  "solver_time" : solver_time}

    all_metrics.append(accum_info)

    opened_file.close()

metric_df = pd.DataFrame(all_metrics)

def highlight_rows(x):

    if x.exp_name == 'baseline':
        return['background-color: pink']*5
    else:
        return['background-color: blue']*5

metric_df.style.apply(highlight_rows, axis=0)

# metric_df.to_csv('cur_metrics.csv')
with open('cur_metrics.tex', 'w') as tf:
    tf.write(metric_df.style.to_latex())
# metric_df.style.to_latex()

# display(metric_df.style.apply(highlight_rows, axis=0))
metric_df
# metric_df.exp_name[2]


,exp_name,epe3d,acc3d_strict,acc3d_relax,angle_error,outlier,solver_time
0,forward_flow,0.092443,0.697132,0.874634,0.210007,0.125563,10.344284
1,visibility_smoothness,0.082483,0.565055,0.805950,0.101770,0.195077,16.699733
2,smoothness,0.097475,0.593866,0.820662,0.208463,0.179338,16.613856
3,forward_and_visibility,0.077181,0.614720,0.861715,0.096689,0.138285,9.628811
4,allec,0.093097,0.382482,0.655555,0.057583,0.345538,15.728599
5,baseline,0.116202,0.586211,0.797746,0.357751,0.461535,16.009291
